In [3]:
import torch
from local_util import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)
alexnet.classifier[6] = torch.nn.Linear(4096, 10).to(device)

alexnet_pcn = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)
alexnet_pcn.classifier = PCN([9216, 4096, 4096, 10], dimensions=16).to(device)


Using cache found in C:\Users\hette/.cache\torch\hub\pytorch_vision_v0.10.0
Using cache found in C:\Users\hette/.cache\torch\hub\pytorch_vision_v0.10.0


In [4]:
m = alexnet.features
alexnet_conv_param_count = 0
for param in m.parameters():
    alexnet_conv_param_count += np.prod(param.shape)

m = alexnet.classifier
alexnet_linear_param_count = 0
for param in m.parameters():
    alexnet_linear_param_count += np.prod(param.shape)

m = alexnet_pcn.features
pcn_alexnet_conv_param_count = 0
for param in m.parameters():
    pcn_alexnet_conv_param_count += np.prod(param.shape)

m = alexnet_pcn.classifier
pcn_alexnet_linear_param_count = 0
for param in m.parameters():
    pcn_alexnet_linear_param_count += np.prod(param.shape)
pcn_alexnet_linear_param_count -= 9216

In [6]:
print(f'fcn alexnet linear layers: {alexnet_linear_param_count}')
print(f'fcn alexnet total layers: {alexnet_conv_param_count + alexnet_linear_param_count}')

print(f'pcn alexnet linear layers: {pcn_alexnet_linear_param_count}')
print(f'pcn alexnet total layers: {pcn_alexnet_conv_param_count + pcn_alexnet_linear_param_count}')

fcn alexnet linear layers: 54575114
fcn alexnet total layers: 57044810
pcn alexnet linear layers: 286890
pcn alexnet total layers: 2756586


In [18]:
import torchsummary
torchsummary.summary(alexnet, (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25